# Lab 04

In this lab, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3


In [1]:
import pandas as pd
import numpy as np
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

# printing the dates dataframe w/ pandas Series

In [2]:
dates = ["•04/20/2009; 04/20/09; 4/20/09; 4/3/09; 9/36/308",
"•Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;",
"•20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009",
"•Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009",
"•Feb 2009; Sep 2009; Oct 2010",
"•6/2008; 12/2009",
"•2009; 2010"]

dates = pd.Series(dates)
dates

0     •04/20/2009; 04/20/09; 4/20/09; 4/3/09; 9/36/308
1    •Mar-20-2009; Mar 20, 2009; March 20, 2009; Ma...
2    •20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 ...
3      •Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
4                        •Feb 2009; Sep 2009; Oct 2010
5                                     •6/2008; 12/2009
6                                          •2009; 2010
dtype: object

In [3]:
dates = ["•04/20/2009;", "04/20/09;", "4/20/09;", "4/3/09;",
"•Mar-20-2009;", "Mar 20, 2009;", "March 20, 2009;", "Mar. 20, 2009;", "Mar 20 2009;", "October 14 1974",
"•20 Mar 2009;", "20 March 2009;", "20 Mar. 2009;", "20 March, 2009","2June, 1999",
"•Mar 20th, 2009;", "Mar 21st, 2009;", "Mar 22nd, 2009",
"•Feb 2009;", "Sep 2009;", "Oct 2010",
"•6/2008;", "12/2009",
"•2009;", "2010"]

dates = pd.Series(dates)
dates

0         •04/20/2009;
1            04/20/09;
2             4/20/09;
3              4/3/09;
4        •Mar-20-2009;
5        Mar 20, 2009;
6      March 20, 2009;
7       Mar. 20, 2009;
8         Mar 20 2009;
9      October 14 1974
10       •20 Mar 2009;
11      20 March 2009;
12       20 Mar. 2009;
13      20 March, 2009
14         2June, 1999
15    •Mar 20th, 2009;
16     Mar 21st, 2009;
17      Mar 22nd, 2009
18          •Feb 2009;
19           Sep 2009;
20            Oct 2010
21            •6/2008;
22             12/2009
23              •2009;
24                2010
dtype: object

# To create the pattern 1

In [4]:
## format in pattern 1 is  month/date/year, month in Digits just like:  "•04/20/2009;", "04/20/09;"

pattern1 = r'(0?[1-9]|1[0-2])[\/\-](0?[1-9]|[12]\d|30|31)[\/\-](\d{4}|\d{2})'
#cond_pattern1 = dates.str.contains(pattern1)
#re.findall(pattern1, dates)
# dates.str.contains(pattern1)
# dates.str.findall(pattern1)
df1 = dates.str.extractall(pattern1)
df1.columns = ["month", "day", "year"]
df1 = df1.reset_index()
df1

,level_0,match,month,day,year
0,0,0,04,20,2009
1,1,0,04,20,09
2,2,0,4,20,09
3,3,0,4,3,09


# To create the pattern 2

In [5]:
## format in pattern 2 is month...date...year, month in Letter  "Mar-20-2009;", "Mar 20, 2009; or October 14 1974, Mar 20th, 2009;", "Mar 21st, 2009;", "Mar 22nd, 2009",
   
pattern2 = r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.]*[ -](\d{1,2})[a-z\.\,]*[ -](\d{4})'
df2=dates.str.extractall(pattern2)
df2.columns = ["month", "day", "year"]
df2 = df2.reset_index()
df2

,level_0,match,month,day,year
0,4,0,Mar,20,2009
1,5,0,Mar,20,2009
2,6,0,Mar,20,2009
3,7,0,Mar,20,2009
4,8,0,Mar,20,2009
5,9,0,Oct,14,1974
6,15,0,Mar,20,2009
7,16,0,Mar,21,2009
8,17,0,Mar,22,2009


In [6]:
df1.match.value_counts()

0    4
Name: match, dtype: int64

# To create the pattern 3

In [7]:
## format in pattern 3 is  date...month...year, month in Letter , date is optional just like: "20 Mar 2009;" or  "Oct 2010"

pattern3 = r'(\d{1,2})?[ -]?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.\,]*[ -](\d{4})'
#cond_pattern3 = dates.str.contains(pattern3)
#dates.str.findall(pattern3)
df3=dates.str.extractall(pattern3)
df3.columns = ["day", "month", "year"]
df3 = df3.reset_index()
df3

,level_0,match,day,month,year
0,10,0,20,Mar,2009
1,11,0,20,Mar,2009
2,12,0,20,Mar,2009
3,13,0,20,Mar,2009
4,14,0,2,Jun,1999
5,18,0,NaN,Feb,2009
6,19,0,NaN,Sep,2009
7,20,0,NaN,Oct,2010


In [8]:
df3.shape

(8, 5)

# To create the pattern 4

In [9]:
## format in pattern 4 is month/year, month in Digits just like : "6/2008;", "12/2009"

pattern4 = r'(\d{1,2})[/](\d{4})'

df4 = dates.str.extractall(pattern4)
df4.insert(0, column='day', value=np.nan)
df4.columns = ["day" , "month", "year"]
df4 = df4.reset_index()
df4

,level_0,match,day,month,year
0,0,0,NaN,20,2009
1,21,0,NaN,6,2008
2,22,0,NaN,12,2009


# To create the pattern 5

In [10]:
## format in pattern 5 is  year, must be 4 Digits just like : "2002;", "2009"

pattern5 = r'(\d{4})'
#cond_pattern5 = dates.str.contains(pattern5)
df5 = dates.str.extractall(pattern5)
df5.insert(0, column='day', value=np.nan)
df5.insert(1, column='month', value=np.nan)
df5.columns = ["month", "day", "year"]
df5 = df5=df5.reset_index()
df5

,level_0,match,month,day,year
0,0,0,NaN,NaN,2009
1,4,0,NaN,NaN,2009
2,5,0,NaN,NaN,2009
3,6,0,NaN,NaN,2009
4,7,0,NaN,NaN,2009
5,8,0,NaN,NaN,2009
6,9,0,NaN,NaN,1974
7,10,0,NaN,NaN,2009
8,11,0,NaN,NaN,2009
9,12,0,NaN,NaN,2009


In [11]:
output = df1.append(df2[~df2.level_0.isin(df1.level_0)])

In [12]:
df2[~df2.level_0.isin(df1.level_0)]

,level_0,match,month,day,year
0,4,0,Mar,20,2009
1,5,0,Mar,20,2009
2,6,0,Mar,20,2009
3,7,0,Mar,20,2009
4,8,0,Mar,20,2009
5,9,0,Oct,14,1974
6,15,0,Mar,20,2009
7,16,0,Mar,21,2009
8,17,0,Mar,22,2009


In [13]:
output.shape

(13, 5)

In [14]:
output = output.append(df3[~df3.level_0.isin(output.level_0)])

In [15]:
output.shape

(21, 5)

In [16]:
output = output.append(df4[~df4.level_0.isin(output.level_0)])

In [17]:
output.shape

(23, 5)

In [18]:
output = output.append(df5[~df5.level_0.isin(output.level_0)])

In [19]:
output.shape

(25, 5)

In [25]:
output = pd.DataFrame(output,columns = ["level_0", "match", "day", "month","year"])
output

,level_0,match,day,month,year
0,0,0,20,04,2009
1,1,0,20,04,1909
2,2,0,20,4,1909
3,3,0,3,4,1909
0,4,0,20,Mar,2009
1,5,0,20,Mar,2009
2,6,0,20,Mar,2009
3,7,0,20,Mar,2009
4,8,0,20,Mar,2009
5,9,0,14,Oct,1974


In [24]:
output.year = np.where(output.year.apply(len)==2, "19"+output.year, output.year)
output = output.fillna("1")

In [27]:
month_replace ={
        'Jan' : 1,
        'Feb' : 2,
        'Mar' : 3,
        'Apr' : 4,
        'May' : 5,
        'Jun' : 6,
        'Jul' : 7,
        'Aug' : 8,
        'Sep' : 9, 
        'Oct' : 10,
        'Nov' : 11,
        'Dec' : 12
}

In [28]:
output.month = output.month.replace(month_replace)

In [29]:
output.day = output.day.astype(int)
output.month = output.month.astype(int)
output.year = output.year.astype(int)

In [30]:
output["date"] = pd.to_datetime(output.loc[:,["year", "month", "day"]])

output = output.sort_values(["date", "level_0"]).reset_index(drop=True)

In [31]:
output.level_0

0      3
1      1
2      2
3      9
4     14
5     21
6     23
7     18
8      4
9      5
10     6
11     7
12     8
13    10
14    11
15    12
16    13
17    15
18    16
19    17
20     0
21    19
22    22
23    24
24    20
Name: level_0, dtype: int64

# the function to print the sorted dataframe

In [34]:
def date_sorter():
    
    import numpy as np
    import re

  
    # 1) month/date/year, month in Digits just like:  "•04/20/2009;", "04/20/09;"==========================
    pattern1 = r'(0?[1-9]|1[0-2])[\/\-](0?[1-9]|[12]\d|30|31)[\/\-](\d{4}|\d{2})'
    df1 = df.str.extractall(pattern1)
    df1.columns = ["month", "day", "year"]
    df1 = df1.reset_index()
    #df1
    
    # 2) month...date...year, month in Letter  "Mar-20-2009;", "Mar 20, 2009; or October 14 1974, Mar 20th, 2009;", "Mar 21st, 2009;", "Mar 22nd, 2009",
    pattern2 = r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.]*[ -](\d{1,2})[a-z\.\,]*[ -](\d{4})'
    df2=df.str.extractall(pattern2)
    df2.columns = ["month", "day", "year"]
    df2 = df2.reset_index()
    #df2
    
    #3) date...month...year, month in Letter , date is optional just like: "20 Mar 2009;" or  "Oct 2010"==============
    pattern3 = r'(\d{1,2})?[ -]?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.\,]*[ -](\d{4})'
    df3=df.str.extractall(pattern3)
    df3.columns = ["day", "month", "year"]
    df3 = df3.reset_index()
    #df3
    
    
    # 4) month/year, month in Digits just like : "6/2008;", "12/2009"==============================
    pattern4 = r'(\d{1,2})[/](\d{4})'
    df4 = df.str.extractall(pattern4)
    df4.insert(0, column='day', value=np.nan)
    df4.columns = ["day" , "month", "year"]
    df4 = df4.reset_index()
    #df4
    
    
    ## 5) year, must be 4 Digits just like : "2002;", "2009"=================================================
    pattern5 = r'(\d{4})'
    df5 = df.str.extractall(pattern5)
    df5.insert(0, column='day', value=np.nan)
    df5.insert(1, column='month', value=np.nan)
    df5.columns = ["month", "day", "year"]
    df5 = df5=df5.reset_index()
    #df5
    
    output = df1.append(df2[~df2.level_0.isin(df1.level_0)])
    #output.shape
    output = output.append(df3[~df3.level_0.isin(output.level_0)])
    #output.shape
    output = output.append(df4[~df4.level_0.isin(output.level_0)])
    #output.shape
    output = output.append(df5[~df5.level_0.isin(output.level_0)])
    #output.shape
    
    output = pd.DataFrame(output,columns = ["level_0", "match", "day", "month","year"])
    output.year = np.where(output.year.apply(len)==2, "19"+output.year, output.year)
    output = output.fillna("1")
    
    month_replace ={
            'Jan' : 1,
            'Feb' : 2,
            'Mar' : 3,
            'Apr' : 4,
            'May' : 5,
            'Jun' : 6,
            'Jul' : 7,
            'Aug' : 8,
            'Sep' : 9, 
            'Oct' : 10,
            'Nov' : 11,
            'Dec' : 12
    }
    
    output.month = output.month.replace(month_replace)
    
    output.day = output.day.astype(int)
    output.month = output.month.astype(int)
    output.year = output.year.astype(int)
    
    output["date"] = pd.to_datetime(output.loc[:,["year", "month", "day"]])
    
    output = output.sort_values(["date", "level_0"]).reset_index(drop=True)
    #output.info()
    #return_value = pd.Series(output.level_0, name="index")
    
    return output.level_0#return_value # Your answer here

In [35]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Name: level_0, Length: 500, dtype: int64

In [ ]:
I used 5 regex to extract the date, note that ORDER is important:
1) month/date/year, month in Digits
2) month...date...year, month in Letter
3) date...month...year, month in Letter , date is optional
4) month/year, month in Digits
5) year, must be 4 Digits
Note that if year in 5) and 4) are different, use 5) instead.
Before submitting, check if the year, month, and date you get is in the correct range using:
df1[~df0[2].isin(range(1900, 2017))]
df1[~df0[0].isin(range(1,